In [1]:
from dotenv import load_dotenv
from syncflowai import store_processed_mission, store_raw_response, store_feedback, generate_feedback, extract_mission_details, generate_mission
import datetime

load_dotenv()

True

In [18]:
current_date = datetime.datetime.now() + datetime.timedelta(days=10)
mission_count = 0
total_days = 15  # Durée totale en jours
missions_per_day = 7  # Missions par jour

for i in range(total_days * missions_per_day):
    print(f"Mission {i+1}", end="\r")
    mission = generate_mission()
    print(f"Extracting details for mission {i+1}")
    _, raw_response = extract_mission_details(mission)
    
    # Calcul de la limite inférieure de la note en fonction du jour
    day_number = (current_date - (datetime.datetime.now() + datetime.timedelta(days=10))).days
    lower_rating_limit = min(5, 1 + day_number // (total_days // 4))
    
    print(f"Generating feedback for mission {i+1}")
    feedback = generate_feedback(mission = mission, raw_response = raw_response, current_date = current_date, rating=)
    print(lower_rating_limit)
    print(f"Storing data for mission {i+1}")
    store_raw_response(raw_response)
    store_feedback(feedback)

    mission_count += 1
    if mission_count >= missions_per_day:
        current_date += datetime.timedelta(days=1)
        mission_count = 0
        print(current_date)

KeyboardInterrupt: 

In [3]:
mission = generate_mission()

In [4]:
details = extract_mission_details(mission)

In [5]:
details[0]

{'name': 'Web Application Security Enhancement',
 'abstract': 'Strengthening the security and performance of a web application, with and without specific technology requirements.',
 'detail': 'The primary objective is two-fold: <b>Penetration Testing</b> of the web application to identify and mitigate vulnerabilities, and performance enhancement. <br><br>For the mission with specific technology preferences, the role demands expertise in <b>Penetration Testing</b> with a mandatory skill in <b>Cassandra database</b> assessment. Comprehensive security evaluation encompassing system, network, and potential social engineering threats is expected. <br><br>For the mission without technological preference, a strategic plan for performance and security enhancement is sought, with open choices in technology stacks. The focus is on creating a system that is not only efficient and robust but also secured against cyber threats.',
 'roles': [{'role_name': '🔍 Penetration Tester',
   'skills_required'

In [41]:
import random
import datetime
def generate_ratings_list():
    # Créer une liste de 150 notes avec une moyenne initiale de 3.5
    # Pour garantir une moyenne de 3.5, nous pouvons utiliser une combinaison de 3 et 4
    ratings = [3 if i < 75 else 4 for i in range(150)]

    # Ajuster aléatoirement les valeurs tout en maintenant la moyenne
    for _ in range(1000):  # Nombre arbitraire d'itérations pour l'ajustement
        idx1, idx2 = random.sample(range(150), 2)
        if ratings[idx1] > 1 and ratings[idx2] < 5:
            ratings[idx1] -= 1
            ratings[idx2] += 1

    return ratings

# Générer les notes
ratings = generate_ratings_list()

# Vérifier la moyenne
average = sum(ratings) / len(ratings)
print(f"Moyenne des notes: {average}")
print(f"Notes: {ratings}")

Moyenne des notes: 3.5
Notes: [3, 5, 4, 4, 3, 5, 3, 1, 1, 3, 3, 3, 3, 4, 5, 3, 3, 2, 5, 5, 3, 3, 4, 4, 4, 4, 4, 1, 5, 2, 2, 2, 3, 3, 5, 2, 4, 2, 1, 4, 3, 1, 5, 5, 3, 1, 3, 1, 3, 3, 3, 5, 4, 4, 1, 3, 3, 5, 4, 1, 4, 1, 5, 1, 5, 2, 5, 5, 4, 3, 5, 5, 5, 5, 4, 5, 3, 5, 4, 4, 2, 5, 5, 3, 5, 3, 4, 3, 5, 2, 1, 5, 5, 4, 3, 5, 1, 5, 5, 5, 5, 3, 5, 3, 4, 4, 2, 4, 5, 5, 2, 2, 4, 4, 5, 1, 1, 5, 2, 2, 5, 5, 4, 5, 3, 2, 1, 2, 1, 2, 5, 5, 1, 2, 4, 2, 5, 5, 3, 5, 1, 4, 5, 4, 5, 5, 5, 5, 5, 4]


In [2]:
import random
import datetime
def generate_ratings_list():
    # Créer une liste de 150 notes avec une moyenne initiale de 3.5
    # Pour garantir une moyenne de 3.5, nous pouvons utiliser une combinaison de 3 et 4
    ratings = [3 if i < 75 else 4 for i in range(150)]

    # Ajuster aléatoirement les valeurs tout en maintenant la moyenne
    for _ in range(1000):  # Nombre arbitraire d'itérations pour l'ajustement
        idx1, idx2 = random.sample(range(150), 2)
        if ratings[idx1] > 1 and ratings[idx2] < 5:
            ratings[idx1] -= 1
            ratings[idx2] += 1

    return ratings

# Générer les notes
ratings = generate_ratings_list()

# Vérifier la moyenne
average = sum(ratings) / len(ratings)
print(f"Moyenne des notes: {average}")
print(f"Notes: {ratings}")

# Liste des ratings
ratings_list = generate_ratings_list()


Moyenne des notes: 3.5
Notes: [5, 1, 1, 4, 5, 4, 4, 2, 5, 5, 3, 1, 4, 4, 5, 1, 5, 2, 4, 4, 4, 3, 5, 2, 3, 4, 5, 4, 3, 5, 1, 3, 5, 5, 3, 5, 3, 3, 1, 1, 5, 4, 5, 5, 1, 2, 2, 5, 4, 1, 1, 4, 3, 1, 1, 4, 4, 2, 5, 5, 5, 4, 5, 1, 3, 3, 2, 5, 1, 5, 5, 4, 4, 5, 3, 2, 4, 3, 4, 4, 5, 5, 5, 4, 5, 2, 5, 4, 5, 5, 5, 2, 4, 5, 4, 5, 3, 2, 5, 1, 1, 3, 4, 5, 4, 4, 2, 2, 4, 4, 4, 5, 4, 5, 3, 2, 3, 1, 3, 5, 4, 1, 4, 3, 2, 2, 4, 5, 3, 5, 2, 3, 4, 4, 2, 4, 3, 5, 4, 3, 3, 2, 4, 5, 5, 1, 3, 4, 5, 4]
Date: 2023-12-09
Extracting details for mission 1 on 2023-12-09
Generating feedback for mission 1 on 2023-12-09
Rating for mission 1: 5
Storing data for mission 1 on 2023-12-09
Extracting details for mission 2 on 2023-12-09
Generating feedback for mission 2 on 2023-12-09
Rating for mission 2: 2
Storing data for mission 2 on 2023-12-09
Extracting details for mission 3 on 2023-12-09
Generating feedback for mission 3 on 2023-12-09
Rating for mission 3: 2
Storing data for mission 3 on 2023-12-09
Extracting details for

14

In [69]:

# Indice pour suivre le rating actuel
rating_index = 0

# Date de début
start_date = datetime.datetime.now() + datetime.timedelta(days=10)
total_days = 2  # Durée totale en jours
missions_per_day = 10  # Missions par jour

for day in range(total_days):
    current_date = start_date 
    print(f"Date: {current_date.date()}")

    for mission_number in range(missions_per_day):
        print(f"Mission {mission_number + 1} on {current_date.date()}", end="\r")
        mission = generate_mission()
        print(f"Extracting details for mission {mission_number + 1} on {current_date.date()}")
        _, raw_response = extract_mission_details(mission)
        
        # Utilisation du rating actuel de la liste
        current_rating = ratings_list[rating_index]

        print(f"Generating feedback for mission {mission_number + 1} on {current_date.date()}")
        feedback = generate_feedback(mission=mission, raw_response=raw_response, feedback_date=current_date, rating=current_rating)
        print(f"Rating for mission {mission_number + 1}: {current_rating}")
        print(f"Storing data for mission {mission_number + 1} on {current_date.date()}")
        store_raw_response(raw_response)
        store_feedback(feedback)

        # Mise à jour de l'indice de rating
        rating_index += 1

    print(f"Finished missions for {current_date.date()}\n")

Date: 2023-12-09
Extracting details for mission 1 on 2023-12-09
Generating feedback for mission 1 on 2023-12-09
Rating for mission 1: 1
Storing data for mission 1 on 2023-12-09
Extracting details for mission 2 on 2023-12-09
Generating feedback for mission 2 on 2023-12-09
Rating for mission 2: 4
Storing data for mission 2 on 2023-12-09
Extracting details for mission 3 on 2023-12-09
Generating feedback for mission 3 on 2023-12-09
Rating for mission 3: 5
Storing data for mission 3 on 2023-12-09
Extracting details for mission 4 on 2023-12-09
Generating feedback for mission 4 on 2023-12-09
Rating for mission 4: 1
Storing data for mission 4 on 2023-12-09
Extracting details for mission 5 on 2023-12-09
Generating feedback for mission 5 on 2023-12-09
Rating for mission 5: 4
Storing data for mission 5 on 2023-12-09
Extracting details for mission 6 on 2023-12-09
Generating feedback for mission 6 on 2023-12-09
Rating for mission 6: 4
Storing data for mission 6 on 2023-12-09
Extracting details for 

APIConnectionError: Connection error.

In [66]:
sum(ratings_list)/len(ratings_list)

3.510204081632653

In [68]:
datetime.datetime.now() + datetime.timedelta(days=9)

datetime.datetime(2023, 12, 9, 15, 13, 6, 753045)

In [62]:
datetime.datetime.today() +datetime.timedelta(days=)

datetime.datetime(2023, 11, 30, 14, 51, 59, 901920)